# SciML SANUM2024
# Lab 3: Reverse-mode automatic differentiation and Zygote.jl

When the number of unknowns becomes large forward-mode automatic differentiation as
implemented in ForwardDiff.jl becomes prohibitively expensive for computing gradients and instead we need to
use reverse-mode automatic differentiation: this is best thought of as implementing the chain-rule
in an automatic fashion, with a specific choice of multiplying the underlying Jacobians.

Computing gradients is important for solving optimisation problems, which is what ultimately what training a neural network
is. Therefore we also look at solving some
simple optimissation problems, using Optimsation.jl

**Learning Outcomes**

1. Computing gradients and derivatives with Zygote.jl
2. Basics of reverse-mode automatic differentiation and pullbacks.
3. Forward-mode automatic differentiation via pushforwards.
4. Using automatic differentiation for implementing gradient descent.
5. Solving optimisation with gradient descent and via Optimsation.jl

## 3.1 Using Zygote.jl for differentiation

We begin with a simple demonstration of Zygote.jl, which can be thought of as a replacement for ForwardDiff.jl that
uses reverse-mode differentiation under the hood. We can differentiate scalar functions, but unlike ForwardDiff.jl it
overloads the `'` syntax to mean differentiation:

In [7]:
using Zygote, Test

f = x -> exp(x^2)
f(0.1), f'(0.1) # "adjoint" is overloaded for functions to mean derivative

@test exp'(1) == exp(1)
@test cos'(1) == -sin(1)

Test Passed

The real power of Zygote.jl is computing gradients (or more generally, Jacobians
of $f : ℝ^m → ℝ^n$ where $n ≪ m$). We can compute a gradient of the function we considered before as follows:

In [17]:
f = function(x)
    ret = zero(eltype(x))
    for k = 1:length(x)-1
        ret += x[k]*x[k+1]
    end
    ret
end

using ForwardDiff



  0.172910 seconds (435.58 k allocations: 782.390 MiB, 19.09% gc time, 41.41% compilation time)
  0.188706 seconds (650.95 k allocations: 45.993 MiB, 1.93% gc time, 66.66% compilation time)


Unlike ForwardDiff.jl, the gradient returns a tuple since multiple arguments are supported in addition
to vector inputs, eg:

In [19]:

gradient((x,y) -> exp(x[1]*cos(y) + x[2]), [0.1,0.2], 0.2)

([1.3203170336021262, 1.3471707570217752], -0.02676415127641921)

Now differentiating this function is not particularly faster than ForwardDiff.jl:

In [ ]:
n = 10_000
x = randn(n)

@time gradient(f, x)[1] # returns a Tuple with a single entry
@time ForwardDiff.gradient(f, x);

It also uses more memory the larger the computation. Take for example
the Taylor series for the exponential from Lab 1:

In [30]:
function exp_t(z, n)
    ret = one(z)
    s = one(z)
    for k = 1:n
        s = s/k * z
        ret = ret + s
    end
    ret
end

@time exp_t(1.0, 1000)
exp_t_1000 = x -> exp_t(x, 10_000)

  0.000005 seconds


#45 (generic function with 1 method)

In [33]:
@time ForwardDiff.derivative(exp_t_1000, 2.0)

  0.000147 seconds (1 allocation: 16 bytes)


7.389056098930649

In [36]:
@time exp_t_1000'(1.0) # Zygotes memory grows with the computation

  0.010836 seconds (250.07 k allocations: 8.185 MiB)


2.718281828459045

The more terms we take the more memory is used, despite the function itself
using no memory:

In [ ]:
#

Another catch is Zygote.jl doesn't support functions that mutate arrays. Here's an example:

In [43]:
function foo(x)
    n = length(x)
    ret = zeros(eltype(x), n)
    for k = 1:n
        ret[k] = x[k] + k
    end
    sum(ret)
end

x = randn(5)
ForwardDiff.gradient(foo,x)
gradient(foo,x)

LoadError: Mutating arrays is not supported -- called setindex!(Vector{Float64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


This is unlike `ForwardDiff.gradient` which works fine for differentiating `f!`.

So why do we use reverse-mode automatic differentiation when it has so many weaknesses
compared to forward-mode?
Because if we write code in just the right way it becomes extremely fast.
For example, if we rewrite `f` in a vectorised form we see a huge improvement over
ForwardDiff.jl:

In [48]:
f = function(x)
    ret = zero(eltype(x))
    for k = 1:length(x)-1
        ret += x[k]*x[k+1]
    end
    ret
end
x = 1:5
f(x)

40

In [166]:
jacobian(x -> x[1:end-1], randn(10))[1]'

10×9 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [170]:
jacobian(x -> x[2:end], randn(10))[1]'

jacobian(broadcast, *, randn(3), randn(3))[2]

3×3 Matrix{Float64}:
 0.107488  0.0       -0.0
 0.0       0.197026  -0.0
 0.0       0.0       -0.110458

In [171]:
jacobian(broadcast, *, randn(3), randn(3))[3]

3×3 Matrix{Float64}:
 0.211531  0.0      -0.0
 0.0       1.00186  -0.0
 0.0       0.0      -0.424847

In [47]:
f2 = x -> sum(x[1:end-1] .* x[2:end]) # a "vectorised" version of above
f2(x)

40

In [55]:
n = 1000
x = randn(n)
@time gradient(f, x); # very slow O(n^2) complexity
@time gradient(f2, x); # very fast O(n) complexity

  0.002140 seconds (26.06 k allocations: 8.699 MiB)
  0.000041 seconds (25 allocations: 65.906 KiB)


In [57]:
n = 1_000_000
x = randn(n)
@time gradient(f2, x); # very fast O(n) complexity

  0.012759 seconds (33 allocations: 61.037 MiB)


**Conclusion**: Zygote.jl is much more brittle, sometimes fails outright, requires
writing functions in a specific way, uses a lot more memory to record complicated operations, but when it works
well it is _extremely_ fast. Thus when we get to neural networks it is paramount that
we design our representations of neural networks in a way that is ameniable to reverse-mode
automatic differentiation, as implemented in Zygote.jl.

------

## 3.2 Pullbacks and back-propagation for scalar functions

We now peek a little under-the-hood to get some intuition on how Zygote.jl is computing
derivatives, and to understand why its so much faster than ForwardDiff.jl in certain situations. Underlying automatic
differentiation in Zygote.jl are so-called "pullback"s. In the scalar
case these are very close to the notion of a derivative. However, rather than
the derivative being a single constant, it's a linear map representing the derivative:
eg, if the derivative of $f(x)$ is denoted $f'(x)$ then the pullback is a linear map
$$
t ↦ f'(x)t.
$$
We can compute pullbacks using the `pullback` routine:

In [61]:
pullback # provided by Zygote
s, p_sin = pullback(sin, 0.1)

(0.09983341664682815, Zygote.var"#75#76"{Zygote.ZBack{ChainRules.var"#sin_pullback#1289"{Float64}}}(Zygote.ZBack{ChainRules.var"#sin_pullback#1289"{Float64}}(ChainRules.var"#sin_pullback#1289"{Float64}(0.9950041652780258))))

`p_sin` contains the map $t ↦ \cos(0.1) t$. Since pullbacks support multiple arguments
it actually returns a tuple with a single entry:

In [65]:
t = 1
@test p_sin(t)[1] == cos(0.1)*t
t = 2
@test p_sin(t)[1] == cos(0.1)*t

Test Passed

Thus to get out the value we use the following:

In [ ]:
#

The reason its a map instead of just a scalar becomes important for the vector-valued case
where Jacobians can often be applied to vectors much faster than constructing the Jacobian matrix and
performing a matrix-vector multiplication.

Pullbacks can be used for determining more complicated derivatives. Consider a composition of three functions
$h ∘ g ∘ f$ where from the Chain Rule we know:
$$
{{\rm d} \over {\rm d} x}[h(g(f(x))] = h'(g(f(x)) g'(f(x)) f'(x)
$$
Essentially we have three pullbacks: the first is the pullback of $f$ evaluated
at $x$, the second corresponding to $g$ evaluated at $f(x)$, and the third
corresponding to $h$ evaluated at $g(f(x))$, that is:
$$
\begin{align*}
 p_1(t) &= f'(x) t  \\
 p_2(t) &= g'(f(x)) t  \\
 p_3(t) &= h'(g(f(x))t
\end{align*}
$$
Thus the derivative is given by either the _forward_ or _reverse_ composition of these functions:
$$
 p_3(p_2(p_1(1))) = p_1(p_2(p_3(1))) = h'(g(f(x))g'(f(x))f'(x).
$$
The first version is called _forward-propagation_ and the second called _back-propagation_.
Forward-propagation is a version of forward-mode automatic differentiation and is essentially equivalent to using dual numbers.
We will see later in the vector case that forward- and back-propagation are not the same,
and that back-propagation is much more efficient provided the output is scalar (or small dimensional).

Let's see pullbacks in action for computing the derivative of $\cos\sqrt{{\rm e}^x}$:

In [80]:
f = exp
g = sqrt
h = cos
(x -> h(g(f(x))))'(0.1)

x = 0.1
y,p₁ = pullback(f, x) # y == f(x), p₁ = t -> f'(x)*t
z,p₂ = pullback(g, y) # z == g(f(x)), p₂ = t -> g'(f(x))*t
w,p₃ = pullback(h, z) # w == h(g(f(x))), p₃ = t -> h'(g(f(x)))*t

@test w == h(g(f(x)))
@test p₁(1)[1] == f'(x)
@test p₂(p₁(1)[1])[1] == g'(f(x))*f'(x)
@test p₃(p₂(p₁(1)[1])[1])[1] ≈ h'(g(f(x)))g'(f(x))*f'(x)

@test p₁(p₂(p₃(1)[1])[1])[1] ≈ h'(g(f(x)))g'(f(x))*f'(x)

Test Passed

In [81]:
# f(x...) is a splat the same as f(x[1], x[2], …, x[end])
@test p₁(p₂(p₃(1)...)...)[1] ≈ h'(g(f(x)))g'(f(x))*f'(x)

Test Passed

We can see how this can lead to an approach for automatic differentiation.
For example, consider the following function composing `sin` over and over:

In [ ]:
# note composition is all we need: x+y is +(x,y)

In [84]:
function manysin(n, x) # sin(sin(…sin(x))) composed n times
    r = x
    for k = 1:n
        r = sin(r)
    end
    r
end

# Why does Zygote use more memory even though this function uses no memory?
n = 5
x =0.1
manysin(n, x)

0.0991753231269109

Now, we would need `n` pullbacks as each time `sin` is called at a different value.
But the number of such pullbacks grows only linearly so this is acceptable. So thus
at a high-level we can think of Zygote as running through and computing all the pullbacks:

In [93]:
pullbacks = Any[] # Any[] means an empty vector whose entries are anything

r = x

for k = 1:n
    r,pₖ = pullback(sin, r) # r changes each time so pullback is cos(r)*t
    push!(pullbacks, pₖ) # add new pullback to end of the vector
end
@test pullbacks[1](2)[1] ≈ 2*cos(0.1)
@test pullbacks[2](2)[1] ≈ 2*cos(sin(0.1))

Test Passed

To deduce the derivative we need can either do forward- or back-propogation by looping through our pullbacks
either in forward- or in reverse-order. Here we implement back-propagation:

In [94]:
r_d = 1
for k = n:-1:1
   r_d = pullbacks[k](r_d)[1]
end
r_d

0.9754309720849006

In [99]:
r_d === (x->manysin(n,x))'(x)

true

Zygote constructs code that is equivalent to this loop automatically,
constructing a high-performance version of this back-propogation loop at compile time using something called source-to-source
differentiation. But there's no getting around the fact that it needs to record the pullbacks so it does use more memory the larger
the computation:

In [ ]:
#

In [100]:
x -> exp(x)

#93 (generic function with 1 method)

In [102]:
manyanons = Any[]
for k = 1:10
    push!(manyanons, x -> exp(k*x))
end
manyanons

10-element Vector{Any}:
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)
 #97 (generic function with 1 method)

------

**Problem 1** Compute the derivative of `manysin` using forward-propagation, by looping through the pull-backs
in the forward direction.

In [ ]:
# TODO: loop through pullbacks in order to compute the derivative.

## 3.3 Pullbacks with multiple arguments

Things become more complicated when we have a function with multiple arguments, even in the
scalar case. Consider now the function $f(g(x), h(x))$. The chain rule tells us that
$$
{{\rm d} \over {\rm d} x}[f(g(x), h(x))] = f_x(g(x), h(x)) g'(x) + f_y(g(x), h(x)) h'(x)
$$
Now we have three pullbacks:
$$
\begin{align*}
p_1(t) &= g'(x) t\\
p_2(t) &= h'(x) t\\
p_3(t) &= [f_x(g(x), h(x))t, f_y(g(x), h(x))t]
\end{align*}
$$
In this case the derivative can be recovered via back-propagation via:
$$
p_1(p_3(1)[1]) + p_2(p_3(1)[2]).
$$
Here we see a simple example:

In [103]:
f = (x,y) -> cos(x*exp(y))
g = sqrt
h = sin
F = x -> f(g(x), h(x))
x = 0.1
F'(x)

-0.7171823264540017

In [112]:
gx,p₁ = pullback(g,x)
hx,p₂ = pullback(h,x)
z,p₃ = pullback(f, gx, hx)
@test z == F(x)
@test p₁(p₃(1)[1])[1] + p₂(p₃(1)[2])[1] == F'(x)

Test Passed

Doing more complicated calculations or indeed algorithms becomes
quite complicated if there are interdependencecies, eg, $f(g(r(x)), h(r(x)))$.
This explains why our first version of a function summing over products of its arguments
was so slow.
Fortunately, there is an alternative: we can focus on composing vector functions.
Eg, such a function can be thought of as composition:
$$
f ∘ 𝐠 ∘ r
$$
where $𝐠(x) = [g(x),h(x)]$. This is a special case of what we discuss in the next section.

------

## 3.4 Gradients and pullbacks

Now we consider computing gradients of functions that are compositions
of vector functions, which neural networks fall into.
Again, we denote the Jacobian as
$$
 J_f = \begin{bmatrix} {∂ f_1 \over ∂x_1} & ⋯ & {∂ f_1 \over ∂x_ℓ} \\
      ⋮ & ⋱ & ⋮ \\
      {∂ f_m \over ∂x_1} & ⋯ & {∂ f_m \over ∂x_ℓ}
\end{bmatrix}
$$
Note that gradients are the transpose of Jacobians: $∇h = J_h^⊤$.
For a scalar-valued function $f : ℝ^n → ℝ$ the pullback represents the linear map
$p_{f,𝐱} : ℝ → ℝ^n$ corresponding to scaling the gradient:
$$
p_{f,𝐱}(t) = J_f(𝐱)^⊤t = ∇f(𝐱) t
$$
Here we see an example:

In [121]:
f = function(𝐱)
    x,y = 𝐱
    exp(x*cos(y))
end
𝐱 = [0.1,0.2]
f_v, p_f = pullback(f, 𝐱)

t = 2
@test p_f(t)[1] == gradient(f, 𝐱)[1]*t

Test Passed

For a function $f : ℝ^n → ℝ^m$ the the pullback represents the linear map $p_{f,𝐱} : ℝ^m → ℝ^n$ given by
$$
p_{f,𝐱}(t) = J_f(𝐱)^⊤𝐭
$$
Here is a simple example:

In [134]:
f = function(𝐱) # map from R^3 to R^2
    x,y,z = 𝐱
    [exp(x*cos(y)*z), cos(x*y+z)]
end
𝐱 = [0.1,0.2,0.3]
f_x, p_f = pullback(f,𝐱)

t = [1,1]
@test p_f(t)[1] == jacobian(f, 𝐱)[1]' * t

Test Passed

BoundsError: BoundsError: attempt to access 20-codeunit String at index [1:21]

BoundsError: BoundsError: attempt to access 20-codeunit String at index [1:21]

BoundsError: BoundsError: attempt to access 4-codeunit String at index [1:5]

Consider a composition $f : ℝ^n → ℝ^m$, $g : ℝ^m → ℝ^ℓ$ and $h : ℝ^ℓ → ℝ$, that is,
we want to compute the gradient of $h ∘ g ∘ f : ℝ^n → ℝ$. The Chain rule tells us that
$$
 J_{h ∘ g ∘ f}(𝐱) = J_h(g(f(𝐱)) J_g(f(𝐱)) J_f(𝐱)
$$
Put another way, the gradiant of $h ∘ g ∘ f$
is given by the transposes of Jacobians:
$$
   ∇[{h ∘ g ∘ f}](𝐱) = J_f(𝐱)^⊤ J_g(f(𝐱))^⊤  ∇h(g(f(𝐱))
$$
Thus we have three pullbacks $p_1 : ℝ^m → ℝ^n$, $p_2 : ℝ^ℓ → ℝ^m$ and $p_3 : ℝ → ℝ^ℓ$ given by
\begin{align*}
 p_1(𝐭) &= J_f(𝐱)^⊤ 𝐭  \\
 p_2(𝐭) &= J_g(f(x))^⊤ 𝐭  \\
 p_3(t) &= ∇h(g(f(𝐱)) t
\end{align*}
The gradient is given by _back-propagation_:
$$
 p_1(p_2(p_3(1))) = J_f(𝐱)^⊤ J_g(f(𝐱))^⊤  ∇h(g(f(𝐱)).
$$
Here the "right" order to do the multiplications is clear: matrix-matrix multiplications are expensive
so its best to do it reverse order so that we only ever have matrix-vector multiplications.
Also, the pullback doesn't give us enough information to implement forward-propagation:
we don't have access to the Jacobian matrices, or their application.

As an example consider computing the gradient of an iteration a simple map like:
$$
𝐟(x,y,z) = \begin{bmatrix} \cos(xy)+z\\ zy-\exp(x)\\ x + y + z \end{bmatrix}
$$
and summing over the result, eg. computing $[1,1,1]^⊤(\underbrace{𝐟 ∘ ⋯ ∘ 𝐟}_{n\hbox{ times}})(𝐱)$.
We implement this with a general function `iteratef`:

In [150]:
𝐟 = function(𝐱)# map from R^3 to R^3
    x,y,z = 𝐱
    [cos(x*y)+z, z*y-sin(x), x+y+z]
end

𝐱 = [0.1,0.2,0.3]
r,p_f = pullback(𝐟, 𝐱)
t = [1,2,3]
p_f(t)[1] == jacobian(𝐟, 𝐱)[1]'*t

function iteratef(𝐱, f, n) # compose f many times and then sum
    for k = 1:n
        𝐱 = f(𝐱)
    end
    sum(𝐱)
end

iteratef(𝐱, 𝐟, 5)

-13.783399764711769

In [152]:
gradient(𝐱 -> iteratef(𝐱, 𝐟, 5), 𝐱)[1]

3-element Vector{Float64}:
 -6.5923400545604744
 43.51048331198635
 86.81831662174272

In [156]:
pullbacks = Any[] # Any[] means an empty vector whose entries are anything

r = 𝐱

for k = 1:n
    r,pₖ = pullback(𝐟, r) # r changes each time so pullback is cos(r)*t
    push!(pullbacks, pₖ) # add new pullback to end of the vector
end

ret,sumpullback = pullback(sum, r) # sum maps R^3 to R

(-13.783399764711769, Zygote.var"#75#76"{Zygote.var"#2989#back#768"{Zygote.var"#762#766"{Vector{Float64}}}}(Zygote.var"#2989#back#768"{Zygote.var"#762#766"{Vector{Float64}}}(Zygote.var"#762#766"{Vector{Float64}}([1.7204985482335622, -12.007870102051191, -3.4960282108941403]))))

To get an idea how this works behind the scenes we can again accumulate the pullbacks:

In [ ]:
𝐱

We can recover the gradient by back-propogation:

In [163]:
r_g = 1

r_g = Vector(sumpullback(r_g)[1])
for k = n:-1:1
    r_g = pullbacks[k](r_g)[1]
end
@test r_g == gradient(𝐱 -> iteratef(𝐱, 𝐟, 5), 𝐱)[1]

Test Passed

In [ ]:
# This explains why reverse-mode is fast: because we only have 
# matrix-vector products

Indeed we match the gradient as computed with Zygote.jl:

In [ ]:
#

**Problem 2** The function `pushforward` represent the map $𝐭 ↦ J_f(𝐱) 𝐭$.
Compute the gradient of `iteratef` as above with forward-mode automatic differentiation by using `pushforward`.
Do so without creating a vector of pushforwards.
Hint: We need to run the pushforward iteration with the identity matrix as the initial value,
but the result of  `pushforward` only works on vectors. So we need to apply it to each column of the matrix manually.

In [ ]:
# TODO: Compute the gradient as above but using pushforward

**Problem 3** Consider a simple forward Euler method approximating the solution to the Pendulum equation with friction:
$$
u'' = τ u' - \sin u
$$
which we can rewrite as a first order system:
$$
\begin{bmatrix}
   u' \\
   v'
   \end{bmatrix} = \begin{bmatrix} v \\ -τ*v - \sin u \end{bmatrix}
$$
That is, we want to implement the iteration
$$
𝐮_{k+1} = 𝐮_k + h*\begin{bmatrix} 𝐮_k[2] \\ -τ 𝐮_k[2] - \sin 𝐮_k[1] \end{bmatrix}
$$
with a specified initial condition $𝐮_0$. For $N = 100$, $h = 0.1$ and $𝐮_0 = [0.1,0.2]$, differentiate
the solution with-respect to $τ$ at $τ = 1$ by creating a vector of pullbacks and implementing back-propagation.
Hint: Forward Euler is a variant of `iteratef` above so you can modify the subsequent pullback construction. Add $τ$ to the vector
of values to capture the relevant dependencies and verify your result by comparing to `gradient`.

## 3.5 Optimisation

A key place where reverse-mode automatic differentiation is essential is large scale optimisation.
As a  simple example we will look at the classic optimisation problem
that solves $A 𝐱 = 𝐛$ where $A$ is symmetric positive definite: find $𝐱$ that minimises
$$
f_{A,𝐛}(𝐱) = 𝐱^⊤ A 𝐱 - 2𝐱^⊤ 𝐛.
$$.
Of course we can use tried-and-true techniques implemented in `\` but here we want
to emphasise we can also solve this with simple optimsation algorithms like gradient desecent
which do not know the structure of the problem. We consider a matrix where we know gradient descent
will converge fast:
$$
A = \begin{bmatrix} 1 & 1/2^α \\ 1/2^α & 1 & ⋱ \\ &  ⋱ & ⋱ & 1/n^α \\ && 1/n^α & 1 \end{bmatrix}
$$
In other words we want to minimise the functional (or the _loss function_)
$$
f_{A,𝐛}(𝐱) = ∑_{k=1}^n x_k^2 + ∑_{k=2}^n x_{k-1} x_k/k^α - ∑_{k=1}^n x_k b_k.
$$
For simplicity we will take $𝐛$ to be the vector with all ones.

Owing to the constraints of Zygote.jl, we need to write this in a vectorised way to ensure Zygote is sufficiently fast.
Here we see that when we do this we can efficiently
compute gradients even
with a million degrees of freedom, way beyond what could ever be done with forward-mode automatic differentiation:

In [175]:
using LinearAlgebra
x = randn(10)
x'x == dot(x,x)

true

In [176]:
n = 5; (2:n) .^ (-1)

4-element Vector{Float64}:
 0.5
 0.3333333333333333
 0.25
 0.2

In [182]:
f = function(x,α)
    n = length(x)
    x'x + 2(x[1:end-1] ./ (2:n) .^ α)'x[2:end] - 2sum(x)
end

#203 (generic function with 1 method)

In [189]:
n = 1_000_000
x = randn(n)
@time gradient(f, x, 2)[1]

  0.127429 seconds (67 allocations: 183.110 MiB, 69.29% gc time)


1000000-element Vector{Float64}:
 -2.156393415904757
 -0.45489682129103626
 -2.297094801172269
 -5.223394704340448
 -2.8524860041976927
 -4.437082578412204
 -0.3137381706675053
  0.7064869386179122
 -2.177277589463353
 -5.94919199181359
 -2.71846722081614
 -0.4697837872474493
 -1.048064933452232
  ⋮
 -3.929148823373159
 -3.493815702849622
 -1.2526140839739681
 -1.805076117190604
 -3.1385027035327067
 -1.5135972040700458
 -0.4539580277886235
 -2.5922998548466567
  5.213551834822463
  0.13930991398250625
 -0.65140110211543
 -2.6549540172874235

For concreteness we first implement our own version of a quick-and-dirty gradient descent:
$$
x_{k+1} = x_k - γ_k ∇f(x_k)
$$
where $γ_k$ is the learning rate. To choose $γ_k$ we just halve
the learning rate until we see decrease in the loss function.

In [191]:
α = 2 # parameter in the functional 
for k = 1:30 # twenty steps of gradient descent
    γ = 1 # learning rate
    y = x - γ*gradient(f, x, α)[1]
    while f(x,α) < f(y,α) # if we aren't decreasing
        γ = γ/2 # halve learning rate
        y = x - γ*gradient(f, x, α)[1]
    end
    x = y # x is the new y
    @show γ, f(x,α)
end


(γ, f(x, α)) = (9.5367431640625e-7, -999998.9059732673)
(γ, f(x, α)) = (1.1920928955078125e-7, -999998.9059732673)
(γ, f(x, α)) = (0.0009765625, -999998.9059732673)
(γ, f(x, α)) = (0.5, -999998.9059732673)
(γ, f(x, α)) = (0.015625, -999998.9059732673)
(γ, f(x, α)) = (0.0078125, -999998.9059732673)
(γ, f(x, α)) = (4.76837158203125e-7, -999998.9059732673)
(γ, f(x, α)) = (2.384185791015625e-7, -999998.9059732673)
(γ, f(x, α)) = (2.9802322387695312e-8, -999998.9059732673)
(γ, f(x, α)) = (1.4901161193847656e-8, -999998.9059732673)
(γ, f(x, α)) = (2.3283064365386963e-10, -999998.9059732673)
(γ, f(x, α)) = (5.820766091346741e-11, -999998.9059732673)
(γ, f(x, α)) = (5.820766091346741e-11, -999998.9059732673)
(γ, f(x, α)) = (5.820766091346741e-11, -999998.9059732673)
(γ, f(x, α)) = (2.9103830456733704e-11, -999998.9059732673)
(γ, f(x, α)) = (2.9103830456733704e-11, -999998.9059732673)
(γ, f(x, α)) = (2.9103830456733704e-11, -999998.9059732673)
(γ, f(x, α)) = (2.9103830456733704e-11, -999998.905

We can compare this with the "true" solution:

In [ ]:
#

In practice its better to use inbuilt optimsation routines and packages. Here we see how we can solve the same problem with
the Optimization.jl package, combined with OptimizationOptimisers.jl that has gradient-based optimisation methods,
in particular `Adam`.

In [192]:
using Optimization # package for optimization
using OptimizationOptimisers # loads gradient-based optimiser, eg Adam

In [199]:
n = 1_000_000

x = randn(n) # initial guess
prob = OptimizationProblem(OptimizationFunction(f, AutoZygote()), x, α)
@time y = solve(prob, Adam(0.03); maxiters=100)
y.u

  5.641426 seconds (22.29 k allocations: 23.285 GiB, 37.73% gc time)


1000000-element Vector{Float64}:
 0.7926794970958331
 0.7686944137269192
 0.5155934708523282
 0.9295595766056008
 0.9336581485775208
 0.9594261131093
 0.9734772204953535
 0.9745894495492411
 0.9720664915796232
 0.9830239649874682
 0.9853045193982235
 0.9810116907962902
 0.9942975618395835
 ⋮
 0.9600657763471684
 1.0094952621275346
 1.005434882999225
 1.0030422125026865
 1.0019575443279658
 0.993896938682554
 1.0006821822489984
 0.9986364887878048
 1.0021922858157455
 0.9936826858773254
 1.0076160939218697
 0.9999614774136861

**Problem 4** This problem considers an example that will connect with  neural networks.
Define ${\rm relu}(x) := \max(0,x)$ and consider an approximation of the form:
$$
p_{𝐚,𝐛}(x) := ∑_{k=1}^n {\rm relu}(a_k x + b_k)
$$
where $𝐚,𝐛 ∈ ℝ^n$. This is a sum of positive convex functions hence consider regression for a positive convex function
like $f(x) =  \exp x$. For $n = 100$,  approximate $𝐚,𝐛$ that minimises $\|p_{𝐚,𝐛}.(𝐱) - f.(𝐱)\|$ where $𝐱$ is a vector containing
100 evenly spaced points between $-1$ and $1$ (inclusive). Compare your regression with $f$ by plotting the two functions.

In [ ]:
# TODO: Construct a model for the function and perform regression using Optimization.jl

In [203]:
myweirdfunction(x::Float64) = x^2
Zygote.pullback(::typeof(myweirdfunction), x) = x, t -> 2x*t

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*